# Goals

This script enables to extract the following information from YSTAFDB:
- processes
- flows
- recycling

In [1]:
import pandas as pd
import os.path

In [2]:
# Define the filepath where the data is stored
data_dir = "../data/Yale/YSTAFDB_CSV_files/"

# Load the reference material list
reference_materials = pd.read_csv(f"{data_dir}reference_materials.csv")

# Get IDs of the materials we are interested in (e.g., nickel)
materials = ['Ni']
material_ids = reference_materials[reference_materials['reference_material'].isin(materials)].reference_material_id.values


In [3]:
# Get lists of processes, flows, recycling, and cross-boundary flows for the specified materials
processes = pd.read_csv(data_dir + "processes.csv")
processes = processes[processes['reference_material_id'].isin(material_ids)].copy()
processes.reset_index(drop=True, inplace=True)

flows = pd.read_csv(data_dir + "flows.csv")
flows = flows[flows['reference_material_id'].isin(material_ids)].copy()
flows.reset_index(drop=True, inplace=True)

recycling = pd.read_csv(data_dir + "recycling.csv")
recycling = recycling[recycling['reference_material_id'].isin(material_ids)].copy()
recycling.reset_index(drop=True, inplace=True)

cross_boundary_flows = pd.read_csv(data_dir + "cross_boundary_flows.csv")
cross_boundary_flows = cross_boundary_flows[cross_boundary_flows['reference_material_id'].isin(material_ids)].copy()
cross_boundary_flows.reset_index(drop=True, inplace=True)

In [4]:
# Get a list of all columns across all dataframes and create a dictionary mapping column names to file names
all_columns = sorted(list(set(processes.columns) | set(flows.columns) | set(recycling.columns) | set(cross_boundary_flows.columns)))
file_names = {x: x.split('_id')[0] for x in all_columns}

In [5]:
# Create a dictionary mapping column names to file names, making modifications to the filenames as needed
cross_ref = set(pd.read_csv(f"{data_dir}indexes.csv").column.unique())
file_names_edited = {}

for col, file in file_names.items():
    if os.path.isfile(data_dir + file + '.csv'):
        file_names_edited[col] = file + '.csv'
        continue
    if os.path.isfile(data_dir + file + 's.csv'):
        file_names_edited[col] = file + 's.csv'
        continue
    if os.path.isfile(data_dir + file + 'es.csv'):
        file_names_edited[col] = file + 'es.csv'
        continue
    if os.path.isfile(data_dir + file[:-1] + 'ies.csv'):
        file_names_edited[col] = file[:-1] + 'ies.csv'
        continue
    if file[-5:] == '_unit':
        file_names_edited[col] = 'units.csv'
        continue
    if file[-6:] == '_label' or file not in cross_ref:
        continue

In [6]:
def get_label(column, values):
    """
    Given a column name and a list of values, return a DataFrame containing the subset of rows from the CSV file that 
    contain any of the specified values in the specified column.

    Parameters:
    column (str): The name of the column to filter on.
    values (list): A list of values to match in the specified column.

    Returns:
    A pandas DataFrame containing the subset of rows from the CSV file that match any of the specified values in the 
    specified column.
    """
    global data_dir
    file_path = data_dir + file_names_edited[column]
    df = pd.read_csv(file_path)
    
    # Extract column name
    if len(column.split('_unit_id')) > 1:
        column = 'unit_id'
    elif len(column.split('_destination')) > 1:
        column = column.split('_destination')[0]
    elif len(column.split('_origin')) > 1:
        column = column.split('_origin')[0]
    
    # Filter rows by given values
    df = df[df[column].isin(values)]
    
    # Select columns to keep
    col_to_keep = column.split('_id')[0] if column != 'publication_id' else 'URL'
    
    # Set index and return selected columns
    return df.set_index(column)[col_to_keep]

In [7]:
def extract_data(file):
    """
    Extracts data from the given file by replacing certain columns with their corresponding labels from reference files.
    
    Args:
        file (pandas.DataFrame): A pandas dataframe containing the data to be extracted.
        
    Returns:
        pandas.DataFrame: A modified version of the original dataframe, where certain columns have been replaced with their corresponding labels.
        
    Raises:
        KeyError: If a column in the input dataframe does not have a corresponding reference file, a KeyError will be raised.
    """
    
    # Create a copy of the input file
    new_file = file.copy()

    # Prepare a list to store the columns that raise KeyError during label extraction
    key_error_cols = []

    # Get the columns that have labels in the database
    cols_with_labels = set(new_file.columns) & set(file_names_edited.keys())

    # Iterate over the columns with labels
    for col in cols_with_labels:
        # Get the set of unique values in the column
        vals = set(new_file[col].unique())

        # Try to get the labels for the values in the column
        try:
            labels = get_label(col, vals)
        except KeyError:
            key_error_cols.append(col)
            continue

        # Map the values to their corresponding labels
        new_file[col] = new_file[col].map(labels)

    # If any columns raised KeyError during label extraction, print a warning message
    if key_error_cols:
        print(f"Warning: could not extract labels for columns {key_error_cols}")

    # Return the modified copy of the input file
    return new_file

In [8]:
# Create an Excel file named 'Nickel_Yale_db.xlsx' and write data from different data frames to different sheets.
output_file = '../data/outputs/Nickel.xlsx'

# Define the sheet names for different data frames.
sheet_names = ['processes', 'flows', 'recycling', 'cross_boundary_flows']

# Write data from each data frame to the corresponding sheet.
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, data in zip(sheet_names, [processes, flows, recycling, cross_boundary_flows]):
        # Extract data from the data frame and write it to the sheet.
        extracted_data = extract_data(data)
        extracted_data.to_excel(writer, sheet_name=sheet_name)